# Data Cleaning
Data cleaning is a crucial step in the data analysis process. It involves identifying and correcting (or removing) errors and inconsistencies in the data to improve its quality. Clean data leads to more accurate analyses and better decision-making. In this notebook, we will perform various data cleaning tasks on our dataset to prepare it for further analysis and modeling.

Our notebook will be structured in a way where we work per column of our dataset. Some columns will not be affected, we have written about this in our accountability report. 

## 0. Loading in dataset

In [1]:
from langdetect import detect, LangDetectException
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from pathlib import Path
import pandas as pd
import numpy as np
import string 
import ast
import re 

file_path = Path("../Data/Raw/Uitgebreide_VKM_dataset.csv")
dataset = pd.read_csv(file_path, low_memory=False)
dataset_before = pd.read_csv(file_path, low_memory=False)

## 0.1 NLP-Function
Here we import our NLP-function we wrote in the NLP.py file. This function will be used to clean the text data in our dataset. We have two versions of the function, a hard and a light version. The hard version does stopword removal, stemming and lemmatization on the text, while the light version only does basic cleaning. We will use the hard version for our cleaning process to ensure that we have the cleanest possible data for our analysis using TF-IDF.

We will use the soft version for our tuned model using sentence embeddings, as the extra steps (stopword removal, stemming and lemmatization) can remove important context from the text that is needed for the embeddings to work properly.

In [2]:
from NLP import hard_nlp, soft_nlp

## 1. name
We will use NLP (Natural Language Processing) on this column. The column is of type string.

In [3]:
# Applying the NLP function to the 'name' column
dataset["name"] = dataset["name"].apply(hard_nlp)

# Check to see if both languages are processed correctly 
dataset["name"].head(15)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/danielvginneken/nltk_data'
    - '/Volumes/External/Development/Projects/School/L2S1LU2-Recommendation/venv/nltk_data'
    - '/Volumes/External/Development/Projects/School/L2S1LU2-Recommendation/venv/share/nltk_data'
    - '/Volumes/External/Development/Projects/School/L2S1LU2-Recommendation/venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


## 2. shortdescription
This whole column will be dropped due to it's similarity with the column "module_tags". We chose to drop this column instead of the module_tags column due to the fact that in places where shortdescription holds on data, module_tags does.

In [ ]:
dataset.drop(columns=["shortdescription"], inplace=True)
dataset.head()

## 3. content
This column is in all rows, except 13 of them, a copy of the description column's data. For the 13 exceptions we will add the data to the description column and then drop the content column after. 

In [ ]:
# Masking where content and description differ
mask_diff = dataset["content"] != dataset["description"]

# Appending content data to description if they differ from eachother. 
def _merge_desc_content(row):
    desc = row["description"]
    cont = row["content"]
    if pd.isna(cont):
        return desc  # Do nothing
    if desc == cont:
        return desc # Dont append if the columns match
    
    # Appending content to description
    return str(desc) + " " + str(cont)

# Merging at places where content and description differ
dataset.loc[mask_diff, "description"] = dataset.loc[mask_diff].apply(
    _merge_desc_content, axis=1
)

# Showing all rows that changed, only their new description and their id value
updated_rows = dataset.loc[mask_diff, ["description", "id"]]
display(updated_rows.values)

# Content data can be dropped after appended to description column
dataset.drop(columns=["content"], inplace=True)

# Single row check
dataset[dataset['id'] == 179].values

## 

## 4. description
After appending the content data to the description on places where the two differ. We will now use NLP.

In [ ]:
# Applying the NLP function to the 'description' column
dataset["description"] = dataset["description"].apply(NLP)

# Check to see if both languages are processed correctly 
print(dataset[dataset['id'] == 315]["description"].values)
print(dataset[dataset['id'] == 234]["description"].values)

## 5. location
Location column is now of datatype string. We will change this to be an array. We do this because sometimes a string is used which says two locations. This is hard to use for filtering, better if its two string of both locations inside of an array. E.g.: "Tilburg & Breda" --> ["Tilburg", "Breda"]

In [ ]:
# Function to get all locations inside an array and for the two 'special cases' to be two string inside of the array representing both locations
def normalize_location(value):
    if pd.isna(value):
        return []

    text = str(value).strip()

    # Special cases
    if text == "Breda en Den Bosch":
        return ["Breda", "Den Bosch"]
    if text == "Den Bosch en Tilburg":
        return ["Den Bosch", "Tilburg"]

    # Default will just convert the single location to be inside of an array for data type consistency
    return [text]

dataset["location"] = dataset["location"].apply(normalize_location)

# Check where both scenarios can be seen
dataset.head(183)

## 6. learningoutcomes
Here we will also use NLP to prepare the data for our model later. However, we also found some inconsistenies inside of the data of this column. You can find values such as: 'ntb', 'nog te bepalen, nan, etc. These values will all have to be properly set to NaN values so that our model later does not take 'nog te bepalen' as an input.

First we will remove the inconsistensies, after we will use NLP.

In [ ]:
# What we need to remove/set to np.nan inside of the learningoutcomes column
error_values_learningoutcomes_contains = ["ntb", "nog niet bekend", "nog te formuleren", "nog nader te bepalen", "nader te bepalen", "nog te bepalen", "n.n.b."]
error_values_learningoutcomes_specific = ["volgt", "nan"]

# Setting all text to lower casing
dataset["learningoutcomes"] = dataset["learningoutcomes"].str.lower()

# Looping and setting values to np.nan for contain errors and specific errors
for val in error_values_learningoutcomes_contains:
    # Only match the full phrase anywhere in the text
    dataset.loc[dataset["learningoutcomes"].str.contains(re.escape(val), na=False), "learningoutcomes"] = np.nan

# Now handle the specific list with exact matches
dataset.loc[dataset["learningoutcomes"].isin(error_values_learningoutcomes_specific), "learningoutcomes"] = np.nan

# Checking is succesful --> number 6, 7, 8 have all been converted to NaN succesfully. Before they were all value: "Nader te bepalen". ALso checked some random locations. 
dataset.head(10)


Now for the NLP part:

In [ ]:
# Applying the NLP function to the 'learningoutcomes' column
dataset["learningoutcomes"] = dataset["learningoutcomes"].apply(NLP)

# Check to see if both languages are processed correctly 
print(dataset[dataset['id'] == 315]["learningoutcomes"].values)
print(dataset[dataset['id'] == 234]["learningoutcomes"].values)

## 7. color-coded columns
We drop the colour columns (Green, Blue, Red, Yellow) as they are not relevant for our recommendation system mainly because it is not useful metadata for understanding user preferences or item characteristics and most values in these columns are Not A Number (NaN).

In [ ]:
# Dropping all 4 columns
dataset.drop(columns=["Rood"], inplace=True)
dataset.drop(columns=["Groen"], inplace=True)
dataset.drop(columns=["Blauw"], inplace=True)
dataset.drop(columns=["Geel"], inplace=True)

# Check
dataset.head()

## 8. module_tags
NLP will once again be used here, but first we need to set all of the empty rows ( [], ['ntb'] ) to np.nan. We do this because these rows provide no information and will mess up the model if we don't correct count them out with np.nan.

In [ ]:
# Convert string representation of lists to actual lists
dataset["module_tags"] = dataset["module_tags"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Now set NaN for empty lists or ['ntb']
dataset.loc[dataset["module_tags"].apply(lambda x: x == [] or x == ['ntb']), "module_tags"] = np.nan

# this position was ['ntb'] before, now we can check to see if it's fixed. Ofcourse we also check on several other locations and other cases like []
dataset.iloc[20]

After properly handling the empty data rows. We now have to get the single string outside of their array and append them into 1 big string so that we can use our NLP function.

In [ ]:
dataset["module_tags"] = dataset["module_tags"].apply(
    lambda x: " ".join(x) if isinstance(x, list) else x
)
dataset["module_tags"].head()

Now for the NLP part:

In [ ]:
# Applying the NLP function to the 'module_tags' column
dataset["module_tags"] = dataset["module_tags"].apply(NLP)

# Check to see if both languages are processed correctly 
print(dataset[dataset['id'] == 315]["module_tags"].values)
print(dataset[dataset['id'] == 234]["module_tags"].values)

## 9. popularity_score
The scores of this column range from 0-500. We will normalize them on a scale from 0-1. 

In [ ]:
# Divide scores by max score
dataset["popularity_score"] = dataset["popularity_score"] / 500

# Check
dataset.head()


## 10. Finalizing

We have completed the data cleaning process for our dataset. The cleaned data is now ready for modeling. We will save the cleaned dataset as a new CSV file to ensure that we can easily access and use it in future steps of our project.

In [ ]:
dataset.to_csv(Path("../Data/Cleaned/cleaned_dataset.csv"), index=False)

This is the final structure and data of our cleaned dataset:

In [ ]:
dataset.head()

This was what it looked like before all the changes we made

In [ ]:
dataset_before.head()

## !!Important mention
We also exported a cleaned dataset where our NLP function was altered to not remove any stopwords, nor perform lemmetization and stemming. This is because sentence embedding works best when you keep closer to original text.